In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from scipy.stats import shapiro
from scipy.stats import levene
import missingno
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
from collections import Counter
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import plot_roc_curve


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


import tensorflow as tf
from tensorflow.keras import Model, Input, backend as K
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
#nlp
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_lg
nlp = en_core_web_lg.load()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))
import nltk
from nltk.stem.porter import *

import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# This Python 3 environment comes with many helpful analytics libraries installed

# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from transformers import TFBertModel, BertConfig, BertTokenizerFast, BertTokenizer

from pathlib import Path


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train =pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test=pd.read_csv('../input/commonlitreadabilityprize/test.csv')
#test=pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
#assigning dataframe as df 
df = train.copy()
dft=test.copy()


**Understanding the data**

In [ ]:

print('SHAPE'.center(80,"*"))
print('rows:{} columns:{} '.format(df.shape[0],df.shape[1]))
print('datatypes'.center(80,"*"))
print(df.dtypes)
print('descriptions'.center(80,"*"))
print(df.describe(include="all"))
print('Info'.center(80,"*"))
print(df.info())

**Columns and null values**

In [ ]:
for colmns in df.columns:
    print('Column name:{}     Null values :{} '.format(colmns,df[colmns].isnull().sum()))
  

**Finding duplicate rows in the data**

In [ ]:

df.duplicated().sum()
df[df.duplicated()]
df.drop_duplicates(keep='first',inplace=True)
df.duplicated().sum()

**Deleting columns having null values**

In [ ]:
dfs=df[['id','excerpt','target','standard_error']]
dfs.info()

**finding whether the target and error has any correlation**

In [ ]:
df['target'].corr(df['standard_error'])

**Remove stopwords and punctuations**

In [ ]:
common_titles= ["MR ", "MRS ", "MS ", "MISS ", "DR ", "HERR ", "MONSIEUR ", "HR ", "FRAU ", "A V M ", "ADMIRAAL ", 
                "ADMIRAL ", "ALDERMAN ", "ALHAJI ", "AMBASSADOR ", "BARON ", "BARONES ", "BRIG ", "BRIGADIER ", "BROTHER ", "CANON ", "CAPT ", "CAPTAIN ", 
                "CARDINAL ", "CDR ", "CHIEF ", "CIK ", "CMDR ", "COL ", "COLONEL ", "COMMANDANT ", "COMMANDER ", "COMMISSIONER ", "COMMODORE ", "COMTE ", 
                "COMTESSA ", "CONGRESSMAN ", "CONSEILLER ", "CONSUL ", "CONTE ", "CONTESSA ", "CORPORAL ", "COUNCILLOR ", "COUNT ", "COUNTESS ", "AIR CDRE ", 
                "AIR COMMODORE ", "AIR MARSHAL ", "AIR VICE MARSHAL ", "BRIG GEN ", "BRIG GENERAL ", "BRIGADIER GENERAL ", "CROWN PRINCE ", "CROWN PRINCESS ", 
                "DAME ", "DATIN ", "DATO ", "DATUK ", "DATUK SERI ", "DEACON ", "DEACONESS ", "DEAN ", "DHR ", "DIPL ING ", "DOCTOR ", "DOTT ", "DOTT SA ", 
                "DR ", "DR ING ", "DRA ", "DRS ", "EMBAJADOR ", "EMBAJADORA ", "EN ", "ENCIK ", "ENG ", "EUR ING ", "EXMA SRA ", "EXMO SR ", "F O ", 
                "FATHER ", "FIRST LIEUTIENT ", "FIRST OFFICER ", "FLT LIEUT ", "FLYING OFFICER ", "FR ", "FRAU ", "FRAULEIN ", "FRU ", "GEN ", "GENERAAL ", 
                "GENERAL ", "GOVERNOR ", "GRAAF ", "GRAVIN ", "GROUP CAPTAIN ", "GRP CAPT ", "H E DR ", "H H ", "H M ", "H R H ", "HAJAH ", "HAJI ", 
                "HAJIM ", "HER HIGHNESS ", "HER MAJESTY ", "HERR ", "HIGH CHIEF ", "HIS HIGHNESS ", "HIS HOLINESS ", "HIS MAJESTY ", "HON ", "HR ", 
                "HRA ", "ING ", "IR ", "JONKHEER ", "JUDGE ", "JUSTICE ", "KHUN YING ", "KOLONEL ", "LADY ", "LCDA ", "LIC ", "LIEUT ", "LIEUT CDR ", 
                "LIEUT COL ", "LIEUT GEN ", "LORD ", "MADAME ", "MADEMOISELLE ", "MAJ GEN ", "MAJOR ", "MASTER ", "MEVROUW ", "MISS ", "MLLE ", "MME ", 
                "MONSIEUR ", "MONSIGNOR ", "MSTR ", "NTI ", "PASTOR ", "PRESIDENT ", "PRINCE ", "PRINCESS ", "PRINCESSE ", "PRINSES ", "PROF ", 
                "PROF DR ", "PROF SIR ", "PROFESSOR ", "PUAN ", "PUAN SRI ", "RABBI ", "REAR ADMIRAL ", "REV ", "REV CANON ", "REV DR ", "REV MOTHER ", 
                "REVEREND ", "RVA ", "SENATOR ", "SERGEANT ", "SHEIKH ", "SHEIKHA ", "SIG ", "SIG NA ", "SIG RA ", "SIR ", "SISTER ", "SQN LDR ", "SR ", 
                "SR D ", "SRA ", "SRTA ", "SULTAN ", "TAN SRI ", "TAN SRI DATO ", "TENGKU ", "TEUKU ", "THAN PUYING ", "THE HON DR ", "THE HON JUSTICE ", 
                "THE HON MISS ", "THE HON MR ", "THE HON MRS ", "THE HON MS ", "THE HON SIR ", "THE VERY REV ", "TOH PUAN ", "TUN ", "VICE ADMIRAL ", 
                "VISCOUNT ", "VISCOUNTESS ", "WG CDR "]

common_titles_new=[]
for i in common_titles:
    common_titles_new.append(i.strip().lower())
    

    
    
    

In [ ]:
#preprocessing the text data 
def swst(x):

    text_sw=[]
    text_final=[]
    
    text_wp=[]
    people_name=[]
    stopwords=list(stop_words) + common_titles_new
    #listing person names
    [people_name.append(w.text) for w in nlp(x).ents if w.label_ == 'PERSON' ]
    [text_wp.append(w) for w in x.split() if w not in people_name]
    text_wp=(" ").join(text_wp)
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for word in text_wp:
        if word in punc:
            text_wp = text_wp.replace(word, "")
    
    
    [text_sw.append(w) for w in (text_wp.lower()).split() if w not in stopwords]
    text_sw=(" ").join(text_sw)
    
            
    stemmer = PorterStemmer()
    for token in text_sw.split():
        text_final.append((stemmer.stem(token)))
    clean_txt=" ".join(text_final)    
        
    return  clean_txt
    

In [ ]:
#traindata
df_train=df[['id','excerpt']]
df_train_target=df[['target']]
df_train['excerpt']=df_train['excerpt'].apply(lambda x: swst(x))
#testdata
df_test=dft[['id','excerpt']]
df_test['excerpt']=df_test['excerpt'].apply(lambda x: swst(x))

In [ ]:
dfs.info()

In [ ]:
import json
#bert_tokenizer
if not os.path.exists('../input/trained-weights/vocab.txt'):
        
        tz = BertTokenizer.from_pretrained("bert-base-cased")
        tz.save_pretrained('./')
        
       
else:
        print('loading the saved pretrained tokenizer')
        tz = BertTokenizerFast.from_pretrained('../input/trained-weights')
#model configuration for bert-base-cased
        
if not os.path.exists('../input/trained-weights/config.json'):
        model_config = BertConfig.from_pretrained('bert-base-cased')
        model_config.output_hidden_states = True
        model_config.save_pretrained('./')
       
else:
        print('loading the saved model_config')
        model_config = BertConfig.from_pretrained('../input/trained-weights')
        model_config.output_hidden_states = True
        


def tok(text):
    
        encoded = tz.encode_plus(
        text=text,  # the sentence to be encoded
        add_special_tokens=True,  # Add [CLS] and [SEP]
        max_length = 256,  # maximum length of a sentence
        pad_to_max_length=True,  # Add [PAD]s
        return_attention_mask = True,  # Generate the attention mask
        truncation=True 
)
        return encoded


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train,df_train_target, test_size=0.2, random_state=42)

In [ ]:

input_ids = []
token_type_ids = []
attention_mask = []
    
for x in X_train['excerpt']:
       
        input_ids.append(tok(x)['input_ids'])
        token_type_ids.append(tok(x)['token_type_ids'])
        attention_mask.append(tok(x)['attention_mask'])
        
x_tr=np.array(input_ids) 
x_tr_id=np.array(token_type_ids)
x_tr_am=np.array(attention_mask)
#making a tuple 
X_train=(x_tr,x_tr_id,x_tr_am)

In [ ]:
input_ids = []
token_type_ids = []
attention_mask = []
    
for x in X_test['excerpt']:
       
        input_ids.append(tok(x)['input_ids'])
        token_type_ids.append(tok(x)['token_type_ids'])
        attention_mask.append(tok(x)['attention_mask'])
        
x_ts=np.array(input_ids) 
x_ts_id=np.array(token_type_ids)
x_ts_am=np.array(attention_mask)
X_test=(x_ts,x_ts_id,x_ts_am)

In [ ]:
input_ids = []
token_type_ids = []
attention_mask = []
    
for x in df_test['excerpt']:
       
        input_ids.append(tok(x)['input_ids'])
        token_type_ids.append(tok(x)['token_type_ids'])
        attention_mask.append(tok(x)['attention_mask'])
        
df_ts=np.array(input_ids) 
df_ts_id=np.array(token_type_ids)
df_ts_am=np.array(attention_mask)
df_val=(df_ts,df_ts_id,df_ts_am)

In [ ]:
#loading and saving tfbert
def load_bert(config):
    if not os.path.exists('../input/trained-weights/tf_model.h5'):
        Path('../output/tfbert-large-uncased').mkdir(parents=True, exist_ok=True)
        bert_model = TFBertModel.from_pretrained("bert-base-cased", config=config)
        bert_model.save_pretrained('./')
    else:
        print('loading the saved pretrained model')
        bert_model = TFBertModel.from_pretrained('../input/trained-weights', config=config)
    return bert_model

In [ ]:
#model building
from keras.utils.vis_utils import plot_model
def build_model(bert_model, max_len=256):    
    input_ids = Input(shape=(256,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(256,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(256,), dtype=tf.int32, name="attention_mask")

    sequence_output = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = Dropout(.1)(clf_output)
    out = Dense(1, activation='linear')(clf_output)
    
    model = Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)
    model.compile(Adam(lr=1e-5), loss='mean_squared_error', metrics=[RootMeanSquaredError(),'accuracy'])
    
    return model


In [ ]:
bert_model=load_bert(model_config)
clf = build_model(bert_model, max_len=256)
clf.summary()

if not os.path.exists('../input/trained-weights/clf.h5'):
        clf.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=4,verbose=1)
        clf.save_weights('./clf.h5')
else:
        print('loading the saved pretrained model')
        clf.load_weights('../input/trained-weights/clf.h5')

predict=clf.predict(df_val).flatten()


In [ ]:
submission = pd.DataFrame()
submission['id'] = df_test['id']
submission['target'] = predict
submission.to_csv("submission.csv",index=False )

In [ ]:
print(pd.read_csv('./submission.csv'))